In [13]:
import sys
sys.path.append("../")

In [23]:
import os
import glob
import fiona
import rasterio
import numpy as np
import rasterio.mask
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow.keras import utils

from tensorflow.python.ops import math_ops
from tensorflow.python.eager import context
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import random_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import gen_image_ops
 
import warnings
warnings.filterwarnings("ignore")

In [44]:
def convert_image_dtype(image, dtype, saturate=False, name=None):
  image = ops.convert_to_tensor(image, name='image')
  dtype = dtypes.as_dtype(dtype)
  if not dtype.is_floating and not dtype.is_integer:
    raise AttributeError('dtype must be either floating point or integer')
  if dtype == image.dtype:
    return array_ops.identity(image, name=name)

  with ops.name_scope(name, 'convert_image', [image]) as name:
    if image.dtype.is_integer and dtype.is_integer:
      scale_in = image.dtype.max
      scale_out = dtype.max
      if scale_in > scale_out:
        scale = (scale_in + 1) // (scale_out + 1)
        scaled = math_ops.floordiv(image, scale)

        if saturate:
          return math_ops.saturate_cast(scaled, dtype, name=name)
        else:
          return math_ops.cast(scaled, dtype, name=name)
      else:
        if saturate:
          cast = math_ops.saturate_cast(image, dtype)
        else:
          cast = math_ops.cast(image, dtype)
        scale = (scale_out + 1) // (scale_in + 1)
        return math_ops.multiply(cast, scale, name=name)
    elif image.dtype.is_floating and dtype.is_floating:
      return math_ops.cast(image, dtype, name=name)
    else:
      if image.dtype.is_integer:
        cast = math_ops.cast(image, dtype)
        scale = 1. / image.dtype.max
        return math_ops.multiply(cast, scale, name=name)
      else:
        scale = dtype.max + 0.5
        scaled = math_ops.multiply(image, scale)
        if saturate:
          return math_ops.saturate_cast(scaled, dtype, name=name)
        else:
          return math_ops.cast(scaled, dtype, name=name)

def adjust_brightness(image, delta):
  with ops.name_scope(None, 'adjust_brightness', [image, delta]) as name:
    image = ops.convert_to_tensor(image, name='image')
    orig_dtype = image.dtype

    if orig_dtype in [dtypes.float16, dtypes.float32]:
      flt_image = image
    else:
      flt_image = convert_image_dtype(image, dtypes.float32)

    adjusted = math_ops.add(
        flt_image, math_ops.cast(delta, flt_image.dtype), name=name)
    return convert_image_dtype(adjusted, orig_dtype, saturate=True)

def random_brightness(image, max_delta, min_delta, seed=None):
    if max_delta < 0 or min_delta <0:
        raise ValueError('max_delta and min_delta must be non-negative.')

    delta = random_ops.random_uniform([], min_delta, max_delta, seed=seed)
    return adjust_brightness(image, delta)

def adjust_hue(image, delta, name=None):
    with ops.name_scope(name, 'adjust_hue', [image]) as name:
        if context.executing_eagerly():
            if delta < -1 or delta > 1:
                raise ValueError('delta must be in the interval [-1, 1]')
        image = ops.convert_to_tensor(image, name='image')
        orig_dtype = image.dtype
        if orig_dtype in (dtypes.float16, dtypes.float32):
            flt_image = image
        else:
            flt_image = convert_image_dtype(image, dtypes.float32)

        rgb_altered = gen_image_ops.adjust_hue(flt_image, delta)
        return convert_image_dtype(rgb_altered, orig_dtype)

def random_hue(image, max_delta, min_delta ,seed=None):
    if max_delta > 0.5:
        raise ValueError('max_delta must be <= 0.5.')

    if max_delta < 0 or min_delta <0:
        raise ValueError('max_delta and min_delta must be non-negative.')

    delta = random_ops.random_uniform([], min_delta, max_delta, seed=seed)
    return adjust_hue(image, delta)

def random_rotate(image, label1, label2, label3, rotation=20):
    rot = np.random.uniform(0, 2*np.pi)
    modified = tfa.image.rotate(image, rot, interpolation='NEAREST')
    m_label1 = tfa.image.rotate(label1, rot, interpolation='NEAREST')
    m_label2 = tfa.image.rotate(label2, rot, interpolation='NEAREST')
    m_label3 = tfa.image.rotate(label3, rot, interpolation='NEAREST')
    return modified, m_label1, m_label2, m_label3

def random_rot_flip(image, label1, label2, label3, width, height):
    m_label1 = tf.reshape(label1, (width, height, 1))
    m_label2 = tf.reshape(label2, (width, height, 1))
    m_label3 = tf.reshape(label3, (width, height, 1))
    axis = np.random.randint(0, 2)
    if axis == 1:
        # vertical flip
        modified = tf.image.flip_left_right(image=image)
        m_label1 = tf.image.flip_left_right(image=m_label1)
        m_label2 = tf.image.flip_left_right(image=m_label2)
        m_label3 = tf.image.flip_left_right(image=m_label3)
    else:
        # horizontal flip
        modified = tf.image.flip_up_down(image=image)
        m_label1 = tf.image.flip_up_down(image=m_label1)
        m_label2 = tf.image.flip_up_down(image=m_label2)
        m_label3 = tf.image.flip_up_down(image=m_label3)
    # rot 90
    k_90 = np.random.randint(4)
    modified = tf.image.rot90(image=modified, k=k_90)
    m_label1 = tf.image.rot90(image=m_label1, k=k_90)
    m_label2 = tf.image.rot90(image=m_label2, k=k_90)
    m_label3 = tf.image.rot90(image=m_label3, k=k_90)

    m_label1 = tf.reshape(m_label1, (width, height))
    m_label2 = tf.reshape(m_label2, (width, height))
    m_label3 = tf.reshape(m_label3, (width, height))
    return modified, m_label1, m_label2, m_label3

def data_augment(image, label1, label2, label3, N_CLASSES):    
    rand1, rand2, rand3, rand4, rand5 = np.random.uniform(size=(5, 1))
    w,h = image.shape[0], image.shape[1]
    status = False
    if rand1 > 0.25:
        status = True
        modified, m_label1, m_label2, m_label3 = random_rot_flip(image, label1, label2, label3, w, h)
        
    if rand2 > 0.25:
        status = True
        try:
            modified, m_label1, m_label2, m_label3 = random_rotate(image, m_label1, m_label2, m_label3, rotation=180)
        except:
            modified, m_label1, m_label2, m_label3 = random_rotate(image, label1, label2, label3, rotation=180)
        
    if rand3 > 0.25:
        status = True
        try:
            modified  = tf.image.random_hue(image, max_delta=0.5)
            m_label1, m_label2, m_label3 = m_label1, m_label2, m_label3
        except:
            modified  = tf.image.random_hue(image, max_delta=0.5)
            m_label1, m_label2, m_label3 = label1, label2, label3
        
    if rand4 > 0.25:
        status = True
        try:
            modified = tf.image.random_brightness(image, max_delta=0.1)
            m_label1, m_label2, m_label3 = m_label1, m_label2, m_label3
        except:
            modified = tf.image.random_brightness(image, max_delta=0.1)
            m_label1, m_label2, m_label3 = label1, label2, label3
            
    if not status:
        modified, m_label1, m_label2, m_label3 = image, label1, label2, label3
        
    m_label1 = tf.cast(m_label1, tf.uint8)
    m_label1 = tf.one_hot(m_label1, depth=N_CLASSES)
    m_label2 = tf.cast(m_label2, tf.uint8)
    m_label2 = tf.one_hot(m_label2, depth=N_CLASSES)
    m_label3 = tf.cast(m_label3, tf.uint8)
    m_label3 = tf.one_hot(m_label3, depth=N_CLASSES)
    
    return modified, m_label1, m_label2, m_label3

class generator_train(utils.Sequence):
    def __init__(self, file_names, batch_size, N_CLASSES, numband):
        self.file_names = file_names
        self.batch_size = batch_size
        self.N_CLASSES = N_CLASSES
        self.numband = numband

    def __len__(self):
        return int(np.floor(len(self.file_names) / self.batch_size))

    def __getitem__(self, index):
        path_segment = self.file_names[index]
        path_img = path_segment.replace('mask_cut_crop','img_cut_crop')
        path_border = path_segment.replace('mask_cut_crop','border_cut_crop')
        path_inner = path_segment.replace('mask_cut_crop','inner_cut_crop')
        
        segment = rasterio.open(path_segment).read().swapaxes(0,1).swapaxes(1,2)
        border = rasterio.open(path_border).read().swapaxes(0,1).swapaxes(1,2)
        inner = rasterio.open(path_inner).read().swapaxes(0,1).swapaxes(1,2)
        image = rasterio.open(path_img).read().swapaxes(0,1).swapaxes(1,2)/255
        image = image[:,:,:self.numband]
        image = tf.cast(image, tf.float32)
        
        image, segment, border, inner = data_augment(image, segment[:,:,0], border[:,:,0], inner[:,:,0] ,self.N_CLASSES)
        segment = tf.image.central_crop(segment, central_fraction=160/256)
        border = tf.image.central_crop(border, central_fraction=160/256)
        inner = tf.image.central_crop(inner, central_fraction=160/256)
        image = tf.image.central_crop(image, central_fraction=160/256)
        image = tf.clip_by_value(image, 0, 1)
        return image, segment, border, inner

    def on_epoch_end(self):
        np.random.shuffle(self.file_names)

class generator_valid(utils.Sequence):
    def __init__(self, file_names, batch_size, N_CLASSES, numband):
        self.file_names = file_names
        self.batch_size = batch_size
        self.N_CLASSES = N_CLASSES
        self.numband = numband

    def __len__(self):
        return int(np.floor(len(self.file_names) / self.batch_size))

    def __getitem__(self, index):
        path_segment = self.file_names[index]
        path_img = path_segment.replace('mask_cut_crop','img_cut_crop')
        path_border = path_segment.replace('mask_cut_crop','border_cut_crop')
        path_inner = path_segment.replace('mask_cut_crop','inner_cut_crop')
        
        segment = rasterio.open(path_segment).read().swapaxes(0,1).swapaxes(1,2)
        border = rasterio.open(path_border).read().swapaxes(0,1).swapaxes(1,2)
        inner = rasterio.open(path_inner).read().swapaxes(0,1).swapaxes(1,2)
        
        image = rasterio.open(path_img).read().swapaxes(0,1).swapaxes(1,2)/255
        image = image[:,:,:self.numband]
        image = tf.cast(image, tf.float32)
        
        segment = tf.cast(segment, tf.uint8)
        segment = tf.one_hot(segment[:,:,0], depth=self.N_CLASSES)
        border = tf.cast(border, tf.uint8)
        border = tf.one_hot(border[:,:,0], depth=self.N_CLASSES)
        inner = tf.cast(inner, tf.uint8)
        inner = tf.one_hot(inner[:,:,0], depth=self.N_CLASSES)
        
        segment = tf.image.central_crop(segment, central_fraction=128/256)
        border = tf.image.central_crop(border, central_fraction=128/256)
        inner = tf.image.central_crop(inner, central_fraction=128/256)
        image = tf.image.central_crop(image, central_fraction=160/256)
        image = tf.clip_by_value(image, 0, 1)
        return image, segment, border, inner

    def on_epoch_end(self):
        np.random.shuffle(self.file_names)
        
def input_fn(file_names, image_size, batch_size, N_CLASSES, numband ,train=True):
    def generator_fn():
        if train:
            generator = utils.OrderedEnqueuer(generator_train(file_names, batch_size, N_CLASSES, numband), False)
        else:
            generator = utils.OrderedEnqueuer(generator_valid(file_names, batch_size, N_CLASSES, numband), False)
        generator.start()
        
        n = 0
        while n<int(len(file_names)):
            image, segment, border, inner = generator.get().__next__()
            yield image, (segment, border, inner)
            n+=1

    output_types = (tf.float32, (tf.float32, tf.float32, tf.float32))
    output_shapes = ((image_size, image_size, numband),
                     ((128, 128, N_CLASSES),
                    (128, 128, N_CLASSES),
                    (128, 128, N_CLASSES)),
                    )

    dataset = tf.data.Dataset.from_generator(generator=generator_fn,
                                             output_types=output_types,
                                             output_shapes=output_shapes)
    dataset = dataset.repeat(1)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def data_gen(mask_path, img_size, batch_size, N_CLASSES, numband, split_ratios, test_data=False, multi=False):
    mask_img = sorted(glob.glob(mask_path))
    np.random.shuffle(mask_img)
    L = len(mask_img)
    if multi:
        if test_data:
            L_train = int(split_ratios*L)
            L_valid = int((1-split_ratios)/2*L)
            L_test = L - L_train - L_valid
            train_dataset = input_fn_multi(mask_img[:L_train], img_size, batch_size, N_CLASSES, numband)
            valid_dataset = input_fn_multi(mask_img[L_train:L_train+L_valid], img_size, batch_size, N_CLASSES, numband, False)
            test_dataset = input_fn_multi(mask_img[L_train+L_valid:L], img_size, batch_size, N_CLASSES, numband, False)
            print("Training:validation:testing = {}:{}:{}".format(L_train, L_valid, L_test))
            return train_dataset, valid_dataset, test_dataset, L_train, L_valid, L_test
        else:
            L_train = int(split_ratios*L)
            L_valid = int(L-L_train)
            train_dataset = input_fn_multi(mask_img[:L_train], img_size, batch_size, N_CLASSES, numband)
            valid_dataset = input_fn_multi(mask_img[L_train:L], img_size, batch_size, N_CLASSES, numband, False)
            print("Training:validation = {}:{}".format(L_train, L_valid))
        return train_dataset, valid_dataset, L_train, L_valid
    else:
        if test_data:
            L_train = int(split_ratios*L)
            L_valid = int((1-split_ratios)/2*L)
            L_test = L - L_train - L_valid
            train_dataset = input_fn(mask_img[:L_train], img_size, batch_size, N_CLASSES, numband)
            valid_dataset = input_fn(mask_img[L_train:L_train+L_valid], img_size, batch_size, N_CLASSES, numband, False)
            test_dataset = input_fn(mask_img[L_train+L_valid:L], img_size, batch_size, N_CLASSES, numband, False)
            print("Training:validation:testing = {}:{}:{}".format(L_train, L_valid, L_test))
            return train_dataset, valid_dataset, test_dataset, L_train, L_valid, L_test
        else:
            L_train = int(split_ratios*L)
            L_valid = int(L-L_train)
            train_dataset = input_fn(mask_img[:L_train], img_size, batch_size, N_CLASSES, numband)
            if L_train == L:
                valid_dataset = input_fn(mask_img[int(L_train*0.8):L], img_size, batch_size, N_CLASSES, numband, False)
                print("Training:validation = {}:{}".format(L_train, L-int(L_train*0.8)))
            else:
                valid_dataset = input_fn(mask_img[L_train:L], img_size, batch_size, N_CLASSES, numband, False)
                print("Training:validation = {}:{}".format(L_train, L_valid))
        return train_dataset, valid_dataset, L_train, L_valid

In [45]:
out_path = '/home/quyet/DATA_ML/Projects/test_building_footprint/tmp'
overlap_mask = os.path.join(out_path, 'mask_cut_crop')
train_dataset, valid_dataset, _, _ = data_gen(os.path.join(overlap_mask, '*.tif'), img_size=160, 
                                                            batch_size=4, N_CLASSES=1, numband=3, 
                                                            split_ratios=0.8, test_data=False, multi=False)

Training:validation = 1771:443


In [6]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size, strides=1, padding='same', 
                use_bias=True, use_batch=True, use_act=True):
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, 
                padding=padding, use_bias=use_bias, kernel_initializer='he_normal', bias_initializer='zeros')(inputs)

    if use_batch:
        x = tf.keras.layers.BatchNormalization()(x)
    
    if use_act:
        x = tf.keras.layers.ELU()(x)

    return x

def up_conv(inputs, filters, use_pool=False):
    if use_pool:
        x = tf.keras.layers.UpSampling2D()(inputs)
    else:
        x = tf.keras.layers.Conv2DTranspose(filters=filters, kernel_size=1, strides=2)(inputs)
    return x


def unet(inputs, filters =[32, 64, 128, 256, 512], out_channel=1):
    x1 = conv_block(inputs, filters[0], 3) #32
    x1_concat = conv_block(x1, filters[0], 3) #32

    x2 = tf.keras.layers.MaxPooling2D(strides=2)(x1_concat)
    x2 = conv_block(x2, filters[1], 3) #64
    x2_concat = conv_block(x2, filters[1], 3) #64

    x3 = tf.keras.layers.MaxPooling2D(strides=2)(x2_concat)
    x3 = conv_block(x3, filters[2], 3) #128
    x3_concat = conv_block(x3, filters[2], 3) #128

    x4 = tf.keras.layers.MaxPooling2D(strides=2)(x3_concat)
    x4 = conv_block(x4, filters[3], 3) #256
    x4_concat = conv_block(x4, filters[3], 3) #256

    x5 = tf.keras.layers.MaxPooling2D(strides=2)(x4_concat)

    bottneck = conv_block(x5, filters[4], 3) #512

    y5 = conv_block(bottneck, filters[4], 3) #512
    
    y4 = up_conv(y5, filters=256, use_pool=True) #256
    y4_concat = tf.keras.layers.Concatenate()([x4_concat, y4])
    y4 = conv_block(y4_concat, filters[3], 3)
    y4 = conv_block(y4, filters[3], 3)
    y4 = conv_block(y4, filters[3], 3)
    
    y3 = up_conv(y4, filters=128, use_pool=True)
    y3_concat = tf.keras.layers.Concatenate()([x3_concat, y3])
    y3 = conv_block(y3_concat, filters[2], 3)
    y3 = conv_block(y3, filters[2], 3)
    y3 = conv_block(y3, filters[2], 3)

    y2 = up_conv(y3, filters=64, use_pool=True)
    y2_concat = tf.keras.layers.Concatenate()([x2_concat, y2])
    y2 = conv_block(y2_concat, filters[1], 3)
    y2 = conv_block(y2, filters[1], 3)
    y2 = conv_block(y2, filters[1], 3)

    y1 = up_conv(y2, filters=32, use_pool=True)
    y1_concat = tf.keras.layers.Concatenate()([x1_concat, y1])
    y1 = conv_block(y1_concat, filters[0], 3)
    y1 = conv_block(y1, filters[0], 3)
    y1 = conv_block(y1, filters[0], 3)

    out = tf.keras.layers.Conv2D(filters = out_channel, kernel_size=1, padding='same',
                                activation='sigmoid')(y1)
    return out

def decoder_block(inputs, name):
    x = conv_block(inputs, 64, 3)
    x = conv_block(x, 32, 3)
    out = tf.keras.layers.Conv2D(filters = 1, kernel_size=1, padding='same',
                                activation='sigmoid', name=name)(x)
    return out

def unet_id(inputs_shape=(160,160,3), filters =[32, 64, 128, 256, 512], out_channel=1):
    inputs = tf.keras.layers.Input(inputs_shape)
    
    unet1 = unet(inputs, filters, out_channel)
    unet2 = unet(inputs, filters, out_channel)
    unet3 = unet(inputs, filters, out_channel)

    out_u1 = tf.keras.layers.Concatenate()([unet1, unet2, unet3])
    out_u2 = tf.keras.layers.Concatenate()([unet1, unet2, unet3])
    out_u3 = tf.keras.layers.Concatenate()([unet1, unet2, unet3])

    segment = decoder_block(out_u1, name='segment')
    segment = tf.keras.layers.Cropping2D(cropping=16,trainable = False)(segment)
    border = decoder_block(out_u2, name='border')
    border = tf.keras.layers.Cropping2D(cropping=16,trainable = False)(border)
    inner = decoder_block(out_u3, name='inner')
    inner = tf.keras.layers.Cropping2D(cropping=16,trainable = False)(inner)

    model = tf.keras.Model(inputs=inputs, outputs=[segment, border, inner])
    return model

# model =  unet_id(inputs_shape=(160,160,3))
# model.summary()

In [47]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true[:,:,:,0])
    y_pred_f = K.flatten(y_pred[:,:,:,0])
    intersection = K.sum(y_true_f * y_pred_f)
    d1 =  (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return d1

def dice_coefficient_loss(y_true, y_pred):
    return 1.-dice_coefficient(y_true, y_pred)

def seg_loss(y_true, y_pred):
    # print(y_true.shape, y_pred.shape)
    dice_s = dice_coefficient_loss(y_true,y_pred)
    
    #ce_loss = BinaryCrossentropy(y_true,y_pred)
    ce_loss =tf.keras.backend.binary_crossentropy(y_true,y_pred)
    
    return ce_loss + dice_s

model =  unet_id(inputs_shape=(160,160,3))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay_rate=1e-15)
model.compile(optimizer = optimizer, loss = {'segment':seg_loss,'border':seg_loss,'inner':seg_loss},
                                    loss_weights={'segment':1.,'border':1.,'inner':1.},
                                    metrics=[
                                        # tfa.metrics.F1Score(num_classes=1, threshold=0.5),
                                            tf.keras.metrics.Precision(thresholds=0.5),
                                            tf.keras.metrics.Recall(thresholds=0.5),
                                            tf.keras.metrics.BinaryAccuracy(name='binary_accuracy', threshold=0.5)])

In [ ]:
from models.callback.save_best import SavebestweightsandEarlyStopping

model_name = 'unet_id'
mission = 'building_footprint'
img_size = 160
num_class = 1 
batch_size = 4

# def lr_decay(epoch):
#     initial_learningrate=1e-3
#     if epoch < 1:
#         return initial_learningrate
#     else:
#         return initial_learningrate * 0.98 ** (epoch)

# if batch_size >1:
#     val_batch_size = int(batch_size/2)
# else:
#     val_batch_size = batch_size

checkpoint_filepath= '/home/quyet/DATA_ML/Projects/segmentation/logs/tmp'
log_dir = '/home/quyet/DATA_ML/Projects/segmentation/logs/graph'
weights_path = '/home/quyet/DATA_ML/WorkSpace/segmentation/weights/%s/'%(model_name) +model_name+'_'+mission+'_'+str(img_size)+'_'+str(num_class)+'class.h5'
patience = 10
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only= True, 
                                                                monitor='val_loss', mode='min', save_best_only=True)
# model_lrscheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=1)
model_lrreduce_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience, min_lr=1e-7, verbose=1)
model_earlystopping_callback = SavebestweightsandEarlyStopping(patience=patience, weights_path=weights_path)
model_endtrainnan_callback = tf.keras.callbacks.TerminateOnNaN()
model_tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)
model_callbacks = [model_checkpoint_callback, 
                   # model_lrscheduler_callback,
                    model_lrreduce_callback, 
                   model_earlystopping_callback,
                    model_tensorboard_callback,]

# model.load_weights('/home/quyet/DATA_ML/WorkSpace/segmentation/weights/msrfnet/msrfnet_road_256_1class_train.h5')
history_train = model.fit(train_dataset, batch_size=batch_size, epochs=100, verbose=1, 
                      callbacks=model_callbacks, validation_data=valid_dataset, 
                      validation_batch_size=val_batch_size, use_multiprocessing=True)


Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/100
    443/Unknown - 1706s 4s/step - loss: 2.2587 - segment_loss: 0.9002 - border_loss: 0.5676 - inner_loss: 0.7908 - segment_precision_2: 0.7093 - segment_recall_2: 0.9277 - segment_binary_accuracy: 0.6826 - border_precision_2: 0.8755 - border_recall_2: 0.9659 - border_binary_accuracy: 0.8499 - inner_precision_2: 0.7522 - inner_recall_2: 0.9866 - inner_binary_accuracy: 0.7459
Save best train weights.
Save best val weights.


2022-07-14 16:40:23.725487: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 566231040 exceeds 10% of free system memory.
2022-07-14 16:40:24.130710: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 566231040 exceeds 10% of free system memory.
2022-07-14 16:40:24.440273: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 566231040 exceeds 10% of free system memory.


443/443 [==============================] - 2129s 5s/step - loss: 2.2587 - segment_loss: 0.9002 - border_loss: 0.5676 - inner_loss: 0.7908 - segment_precision_2: 0.7093 - segment_recall_2: 0.9277 - segment_binary_accuracy: 0.6826 - border_precision_2: 0.8755 - border_recall_2: 0.9659 - border_binary_accuracy: 0.8499 - inner_precision_2: 0.7522 - inner_recall_2: 0.9866 - inner_binary_accuracy: 0.7459 - val_loss: 2.0337 - val_segment_loss: 0.7920 - val_border_loss: 0.4829 - val_inner_loss: 0.7587 - val_segment_precision_2: 0.7446 - val_segment_recall_2: 0.9973 - val_segment_binary_accuracy: 0.7514 - val_border_precision_2: 0.8833 - val_border_recall_2: 1.0000 - val_border_binary_accuracy: 0.8833 - val_inner_precision_2: 0.7671 - val_inner_recall_2: 1.0000 - val_inner_binary_accuracy: 0.7671 - lr: 0.0010

Epoch 00002: LearningRateScheduler setting learning rate to 0.00098.
Epoch 2/100
443/443 [==============================] - ETA: 0s - loss: 2.0971 - segment_loss: 0.8501 - border_loss: 0.

2022-07-14 17:14:37.526490: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 566231040 exceeds 10% of free system memory.
2022-07-14 17:14:37.835307: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 566231040 exceeds 10% of free system memory.


443/443 [==============================] - 2053s 5s/step - loss: 2.0971 - segment_loss: 0.8501 - border_loss: 0.4816 - inner_loss: 0.7654 - segment_precision_2: 0.7046 - segment_recall_2: 0.9909 - segment_binary_accuracy: 0.7017 - border_precision_2: 0.8764 - border_recall_2: 1.0000 - border_binary_accuracy: 0.8764 - inner_precision_2: 0.7521 - inner_recall_2: 0.9988 - inner_binary_accuracy: 0.7516 - val_loss: 1.8739 - val_segment_loss: 0.7358 - val_border_loss: 0.4552 - val_inner_loss: 0.6829 - val_segment_precision_2: 0.7450 - val_segment_recall_2: 0.9961 - val_segment_binary_accuracy: 0.7559 - val_border_precision_2: 0.8797 - val_border_recall_2: 1.0000 - val_border_binary_accuracy: 0.8797 - val_inner_precision_2: 0.7555 - val_inner_recall_2: 1.0000 - val_inner_binary_accuracy: 0.7555 - lr: 9.8000e-04

Epoch 00003: LearningRateScheduler setting learning rate to 0.0009603999999999999.
Epoch 3/100
373/443 [========================>.....] - ETA: 4:16 - loss: 2.1159 - segment_loss: 0.85

In [41]:
import os
import cv2
import glob
import rasterio
import numpy as np
import geopandas as gp
import rasterio.features
import matplotlib.pyplot as plt

import multiprocessing
from functools import partial
from multiprocessing.pool import Pool


# list_geometry = []
def create_mask(shape_id, out_folder):
    # for shape_id in list_shape: 
    with rasterio.open(shape_id.replace('/img_cut_crop_shape/', '/img_cut_crop/').replace('.shp', '.tif')) as src:
        img = src.read()
        height = src.height
        width = src.width
        tr = src.transform
        crs = src.crs
    masks = np.zeros((height, width))

    # print(shape_id)
    shp = gp.read_file(shape_id)
    for i,x in shp.iterrows():
        if not x.geometry==None:
            # print(x.geometry)
            mask = rasterio.features.rasterize([x.geometry] ,out_shape=(height, width) ,transform=tr)
            kernel =  cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
            mask_dil = cv2.dilate(mask, kernel, iterations=1)
            mask_ero = cv2.erode(mask, kernel)

            bound = cv2.subtract(mask_dil, mask_ero)
            mask[bound==1]=2
            mask[mask==255]=1
            masks += mask
            # break

    masks[masks>=2]=2    
    outpath = os.path.join(out_folder, os.path.basename(shape_id).replace('.shp', '.tif'))
    new_dataset = rasterio.open(outpath, 'w', driver='GTiff',
                            height = height, width = width,
                            count = 1, dtype = 'uint8',
                            crs = crs,
                            transform = tr,
                            nodata = 0,
                            compress='lzw')

    new_dataset.write(masks, 1)
    new_dataset.close()

out_folder = '/home/quyet/DATA_ML/Projects/test_building_footprint/aaa/mask_cut_crop'
list_shape = glob.glob('/home/quyet/DATA_ML/Projects/test_building_footprint/aaa/img_cut_crop_shape/*.shp')

if not os.path.exists(out_folder):
        os.mkdir(out_folder)
core = multiprocessing.cpu_count()
p_cnt = Pool(processes=core)
result = p_cnt.map(partial(create_mask,out_folder=out_folder), list_shape)
p_cnt.close()
p_cnt.join()

ERROR 1: PROJ: proj_create_from_database: /home/nghipham/anaconda3/envs/onnx/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/nghipham/anaconda3/envs/onnx/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/nghipham/anaconda3/envs/onnx/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/nghipham/anaconda3/envs/onnx/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/nghipham/anaconda3/envs/onnx/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABA